In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import sys
import re

strPAF_in = '/groups/tanaka/Projects/lungfish-genome/manuscript/Revision_II/winnowmap/full.paf'

reads = dict()
nLines = 0
print(f'Reading the file {strPAF_in}', file=sys.stderr)
with(open(strPAF_in, 'r')) as hFile:
    for line in hFile.readlines():
        readID, readLen, readStart, readEnd, readStrand, targetID, targetLen, targetStart, targetEnd, rest = line.split('\t', 9)
        
        nLines += 1
        if nLines % 10_000_000 == 0:
            print(f'   Read {nLines} lines', file=sys.stderr)
        
        if not reads.get(readID):
            reads[readID] = {'length': int(readLen), 'regions': [], 'chr': set()}
        reads[readID]['regions'].append([int(readStart), int(readEnd)])
        
        # Add the target chromosome to the list
        reads[readID]['chr'].add(targetID)

Reading the file /groups/tanaka/Projects/lungfish-genome/manuscript/Revision_II/winnowmap/full.paf
   Read 10000000 lines
   Read 20000000 lines
   Read 30000000 lines
   Read 40000000 lines
   Read 50000000 lines
   Read 60000000 lines
   Read 70000000 lines
   Read 80000000 lines
   Read 90000000 lines
   Read 100000000 lines
   Read 110000000 lines
   Read 120000000 lines
   Read 130000000 lines
   Read 140000000 lines
   Read 150000000 lines
   Read 160000000 lines


In [ ]:
import pdb


cov_data = []
for readID in reads:
    
    # Merge the regions
    nr_regions = []
    for reg in reads[readID]['regions']:
        for old_reg in nr_regions:
            if (old_reg[0] <= reg[0] <= old_reg[1] 
                   or old_reg[0] <= reg[1] <= old_reg[1]
                   or reg[0] <= old_reg[0] <= reg[1] 
                   or reg[0] <= old_reg[1] <= reg[1]):
                old_reg[0] = min(old_reg[0], reg[0])
                old_reg[1] = max(old_reg[1], reg[1])
                reg = None
                break
        if reg:
            nr_regions.append(reg)

    rc = 0
    for reg in nr_regions:
        rc += (reg[1] - reg[0])
    if rc > reads[readID]['length']:
        print(f'ID: {readID}')
        print(f'Original: {reads[readID]}')
        print(f'Reduced: {nr_regions}')
        break
    cov_data.append(int(rc / reads[readID]['length'] * 100))
    
print(max(cov_data))
print(min(cov_data))
#data_norm = [ min(x, 101) for x in data ]
#plt.hist(data_norm, bins=101)